In [50]:
import requests
import pandas as pd

def get_address_info(address, zip_code):
    api_url = "https://gis.lojic.org/maps/rest/services/LojicSolutions/OpenDataAddresses/MapServer/0/query"
    
    # Define the query parameters to search by FULL_ADDRESS
    params = {
        "where": f"FULL_ADDRESS = '{address}'",
        "outFields": "ZONING_CODE,PARCELID,HISTDIST,MUNI_NAME,FIREDIST,COUNTY",
        "f": "json"
    }
    
    try:
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = response.json()
            features = data.get("features", [])
            if features:
                # Create a DataFrame from the response data
                df = pd.DataFrame([feature["attributes"] for feature in features])
                return df
            else:
                return "Address not found"
        else:
            return "API request failed"
    except Exception as e:
        return str(e)

# Example usage
input_address = "1032 ASH ST"
input_zip_code = "40217"
address_info = get_address_info(input_address, input_zip_code)

if isinstance(address_info, pd.DataFrame):
    print("Address Information:")
    print(address_info)
else:
    print(f"No information found for the address")


Address Information:
  ZONING_CODE      PARCELID  HISTDIST   MUNI_NAME FIREDIST     COUNTY
0          R5  026C01010000         0  LOUISVILLE        4  Jefferson
